# N-BEATS (Darts) ? 1/7/30 kunlik forecast

- Target: rate; past_covariates: qolgan 23 ustun.
- Outlier clip, 70/15/15 split, MAE/MAPE.
- 1/7/30 forecast: covariates oxirgi qiymati bilan uzaytiriladi (agar kelajak ma'lum bo'lmasa).


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# Ma'lumotlarni yuklash
p = Path('datasets/usd_rates_ready.csv')
df = pd.read_csv(p)
df.columns = [c.lower() for c in df.columns]
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

# Target = rate, date ishlatilmaydi, qolgan 23 ta ustun feature sifatida
feature_cols = [c for c in df.columns if c not in ['date', 'rate']]

# Outlierlarni IQR bilan yumshatish (faqat targetda)
def clip_iqr(s, k=1.5):
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    lower, upper = q1 - k * iqr, q3 + k * iqr
    return s.clip(lower, upper)

df['rate'] = clip_iqr(df['rate'])

# Vaqt bo'yicha train/val/test (70/15/15)
n = len(df)
train_end = int(n * 0.70)
val_end = int(n * 0.85)
train_df = df.iloc[:train_end]
val_df = df.iloc[train_end:val_end]
test_df = df.iloc[val_end:]

# Bashorat ufqlari
horizons = [1, 7, 30]

try:
    from darts import TimeSeries
    from darts.models import NBEATSModel
    from darts.dataprocessing.transformers import Scaler
except ImportError:
    raise SystemExit("Darts o'rnatilmagan: pip install darts[u]")

series = TimeSeries.from_dataframe(df, time_col='date', value_cols='rate')
covariates = TimeSeries.from_dataframe(df, time_col='date', value_cols=feature_cols)

scaler_y = Scaler()
scaler_x = Scaler()
series_s = scaler_y.fit_transform(series)
cov_s = scaler_x.fit_transform(covariates)

train, rest = series_s.split_before(0.70)
val, test = rest.split_before(0.50)

cov_train, cov_rest = cov_s.split_before(0.70)
cov_val, cov_test = cov_rest.split_before(0.50)

model = NBEATSModel(
    input_chunk_length=30,
    output_chunk_length=7,
    n_epochs=50,
    random_state=42,
    verbose=False,
)
model.fit(train, past_covariates=cov_train, val_series=val, val_past_covariates=cov_val)

val_pred = model.predict(n=len(val), series=train, past_covariates=cov_train.concatenate(cov_val))
test_pred = model.predict(n=len(test), series=train.concatenate(val), past_covariates=cov_train.concatenate(cov_val).concatenate(cov_test))

mae_val = mean_absolute_error(val.values(), val_pred.values())
mape_val = mean_absolute_percentage_error(val.values(), val_pred.values())
mae_test = mean_absolute_error(test.values(), test_pred.values())
mape_test = mean_absolute_percentage_error(test.values(), test_pred.values())
print(f"Val MAE: {mae_val:.2f}, MAPE: {mape_val:.4f}")
print(f"Test MAE: {mae_test:.2f}, MAPE: {mape_test:.4f}")

# 1/7/30 forecast: covariates oxirgi qiymati bilan uzaytiriladi
horizon_forecasts = {}
for h in horizons:
    future_dates = pd.date_range(df['date'].max() + pd.Timedelta(days=1), periods=h, freq='D')
    last_cov = cov_s.values()[-1]
    future_cov_vals = np.repeat(last_cov[np.newaxis, :], h, axis=0)
    future_cov = TimeSeries.from_times_and_values(future_dates, future_cov_vals)
    full_cov = cov_s.concatenate(future_cov)
    fc = model.predict(n=h, series=series_s, past_covariates=full_cov)
    fc_inv = scaler_y.inverse_transform(fc)
    horizon_forecasts[f'h_{h}'] = fc_inv.pd_dataframe()

horizon_forecasts['h_30'].head()
